In [19]:
import requests
import random
import pandas as pd
from io import StringIO

id_archivo = '1SXWZU0V68kysW3_Slt7aMYERKPK05kM4'
link = f'https://drive.google.com/uc?export=download&id={id_archivo}'

archivo = requests.get(link).text 

lineas = archivo.strip().split('\n')

In [3]:
#Seccionamos el número de almacenes y de clientes.
almacenes, clientes = map(int, lineas[0].split())

indice_capacidades = 0 # Primer *
indice_demandas = 0  # Segundo *
indice_costos_transporte = 0  # Tercer *

capacidad = []
costo_apertura = []
if lineas[1] == '*':
    indice_capacidades = lineas.index("*", 1)  # Primer *
    indice_demandas = lineas.index("*", indice_capacidades + 1)  # Segundo *
    indice_costos_transporte = lineas.index("*", indice_demandas + 1)  # Tercer *
    for i in range(indice_capacidades + 1, indice_demandas):
        linea = lineas[i].split()
        cap = int(float(linea[0]))
        cost= float(linea[1])
        capacidad.append(cap)
        costo_apertura.append(cost)
else:
    for i in range(1, almacenes + 1):
        linea = lineas[i].split()
        cap = int(linea[0])
        cost= float(linea[1])
        capacidad.append(cap)
        costo_apertura.append(cost)

demandas = []
costos_transporte = []
if indice_demandas != 0:
    for i in range(indice_demandas + 1, indice_costos_transporte):
        linea = lineas[i].split()
        demanda_cliente = float(linea[0])
        demandas.append(demanda_cliente)
        costos_transporte.append([])
    almacen = 0
    for i in range(indice_costos_transporte + 1, len(lineas)):
        linea=lineas[i].split()
        cliente_costos = list(map(float, linea))
        costos_transporte[almacen].extend(cliente_costos)
        almacen += 1
else:
    for i in range(almacenes + 1, len(lineas)):
        linea = lineas[i].split()
        if (len(linea)==1):
            demanda_cliente = int(linea[0])
            demandas.append(demanda_cliente)
            costos_transporte.append([])
        else:
            cliente_costos = list(map(float, linea[:almacenes]))
            costos_transporte[-1].extend(cliente_costos)


In [17]:
def algoritmoGenetico():
    #Parámetros del algoritmo genetico
    GENERACIONES = 100
    PROB_CRUCE = 0.6
    PROB_MUTACION = 0.5
    TAMANO_POBLACION = 100

    #Función que crea un individuo, una lista de datos binarios
    def generar_individuo():
        return [random.randint(0, 1) for _ in range(almacenes)]

    #Función que evalua a un individuo
    def evaluar(individuo):
        costo_total_ap = sum(individuo[i] * costo_apertura[i] for i in range(almacenes))
        suma_menor = 0
        menor=0
        bandera = 0
        for i in range (clientes):
            #print("i: ",i)
            for j in range (almacenes):
                #print("j: ",j)
                if individuo[j] == 1 :
                #print(menor,"?-?",costos_transporte[i][j])
                    if bandera == 0:
                        menor = costos_transporte[i][j]
                        bandera = 1
                    if costos_transporte[i][j]<menor:
                        menor = costos_transporte[i][j]
            #print(menor)
            suma_menor += menor
            menor = 0
            bandera = 0
        #print("Individuo: ",individuo,"\nCosto apertura: ", costo_total_ap,"\nCosto transporte: ",suma_menor)
        #print(costo_total_ap + suma_menor)
        return costo_total_ap + suma_menor

    #evaluar(generar_individuo())

    #Función que hace una selección por torneo
    def seleccion_por_torneo(poblacion, k=3):
        seleccionados = random.sample(poblacion, k)
        return min(seleccionados, key=evaluar)

    #Función que hace el cruce
    def cruce_uniforme(ind1, ind2):
        hijo1, hijo2 = ind1[:], ind2[:]
        for i in range(len(ind1)):
            if random.random() < 0.5:
                hijo1[i], hijo2[i] = hijo2[i], hijo1[i]
        return hijo1, hijo2

    #Función que hace la mutación
    def mutar(individuo):
        for i in range(len(individuo)):
            if random.random() < PROB_MUTACION:
                individuo[i] = 1 - individuo[i]  # Cambia entre 0 y 1
        return individuo

    #Se crea la población mediante el uso de "generar_individuo", la cantidad de cuantos individuos son creados la determina el parametro "TAMANO_POBLACION"
    poblacion = [generar_individuo() for _ in range(TAMANO_POBLACION)]

    #En este for vamos a usar las funciones antes implementadas, para encontrar al mejor individuo posible, y de este modo el costo minimo
    for gen in range(GENERACIONES):
        # Evaluar población y seleccionar los mejores
        poblacion = sorted(poblacion, key=evaluar)
        nueva_poblacion = poblacion[:TAMANO_POBLACION // 2]

        # Crear nueva población con cruce y mutación
        while len(nueva_poblacion) < TAMANO_POBLACION:
            # Selección
            padre1 = seleccion_por_torneo(poblacion)
            padre2 = seleccion_por_torneo(poblacion)

            # Cruce
            if random.random() < PROB_CRUCE:
                hijo1, hijo2 = cruce_uniforme(padre1, padre2)
            else:
                hijo1, hijo2 = padre1[:], padre2[:]

            # Mutación
            hijo1 = mutar(hijo1)
            hijo2 = mutar(hijo2)

            # Agregar a la nueva población
            nueva_poblacion.extend([hijo1, hijo2])

        # Reemplazar la población anterior
        poblacion = nueva_poblacion

    # Mejor solución encontrada
    mejor_individuo = min(poblacion, key=evaluar)
    print("Mejor configuración de almacenes:", mejor_individuo)
    print("Costo mínimo:", evaluar(mejor_individuo))

In [18]:
algoritmoGenetico()

500


NameError: name 'random' is not defined

In [6]:
print(almacenes)

500


In [7]:
print(capacidad)
print(costo_apertura)

[369, 534, 737, 414, 1310, 1310, 912, 367, 1140, 924, 1320, 876, 964, 641, 597, 1230, 608, 1040, 1170, 881, 1240, 945, 1040, 1010, 374, 1370, 1020, 1020, 1090, 1130, 1080, 379, 1060, 671, 837, 500, 1310, 952, 1120, 1290, 1090, 925, 922, 589, 895, 980, 1150, 1080, 582, 876, 712, 1400, 460, 1270, 1320, 933, 756, 1240, 1130, 1040, 1190, 1340, 1030, 458, 1200, 1290, 843, 1390, 825, 1130, 648, 1140, 861, 693, 915, 1090, 1250, 515, 1140, 1120, 1140, 1310, 1250, 1220, 955, 1070, 1160, 1010, 842, 1220, 1070, 1270, 1000, 534, 966, 1400, 1310, 673, 1280, 435, 1140, 600, 791, 1380, 1290, 829, 739, 685, 1070, 1150, 970, 1120, 460, 1350, 948, 1030, 1270, 672, 1250, 1030, 905, 1220, 1070, 872, 777, 723, 968, 1250, 1000, 1050, 1340, 952, 529, 650, 559, 1310, 1060, 1100, 1310, 1230, 1030, 1080, 855, 697, 766, 1040, 1110, 1290, 703, 803, 894, 828, 937, 500, 500, 889, 1370, 1420, 918, 1070, 697, 929, 1100, 618, 1330, 1220, 1370, 701, 610, 1280, 1210, 449, 1000, 864, 1230, 792, 1300, 1280, 581, 511, 1330

In [115]:
print(demandas)

[146, 87, 672, 1337, 31, 559, 2370, 1089, 33, 32, 5495, 904, 1466, 143, 615, 564, 226, 3016, 253, 195, 38, 807, 551, 304, 814, 337, 4368, 577, 482, 495, 231, 322, 685, 12912, 325, 366, 3671, 2213, 705, 328, 1681, 1117, 275, 500, 2241, 733, 222, 49, 1464, 222]


In [130]:
print(costos_transporte[1])

[11.3, 1.59, 8.88, 8.3, 6.02, 9.54, 9.51, 6.68, 4.99, 0.66, 4.36, 9.07, 11.3, 5.02, 1.3, 0.472, 7.96, 7.3, 5.25, 2.9, 2.49, 4.29, 4.82, 6.39, 1.84, 4.77, 2.27, 5.11, 2.17, 10.9, 2.16, 6.65, 4.32, 4.98, 7.18, 8.47, 3.94, 7.64, 9.29, 7.45, 4.97, 8.42, 2.22, 7.2, 7.64, 7.92, 3.33, 6.35, 8.07, 6.74, 3.61, 4.5, 7.52, 0.673, 7.86, 3.5, 0.61, 8.83, 9.14, 5.98, 6.37, 5.08, 5.06, 7.68, 6.87, 0.604, 4.1, 7.18, 7.61, 8.09, 4.36, 7.26, 5.31, 7.49, 6.48, 8.82, 7.15, 7.42, 3.34, 3.24, 8.16, 4.54, 6.54, 7.9, 2.45, 4.07, 4.16, 4.47, 1.63, 8.24, 5.16, 4.87, 0.986, 7.13, 7.78, 9.89, 6.14, 1.97, 2.1, 8.39, 4.5, 5.73, 6.31, 6.3, 4.5, 5.82, 5.38, 4.6, 1.76, 2.68, 5.14, 10.1, 8.5, 3.21, 9.36, 8.93, 2.91, 7.15, 2.74, 1.57, 3.18, 0.93, 9.13, 1.56, 6.08, 2.48, 7.04, 0.416, 8.45, 4.99, 4.84, 2.44, 3.39, 10.4, 5.44, 7.36, 3.52, 8.93, 3.58, 7.78, 8.95, 2.64, 5.29, 5.89, 9.04, 3.95, 5.34, 5.66, 2.54, 6.72, 9.28, 8.33, 4.61, 5.34, 10.2, 4.54, 7.27, 8.96, 8.43, 7.61, 7.04, 9.76, 8.33, 7.02, 4.43, 9.46, 9.9, 8.57, 7.

In [103]:
print(archivo)

500	500
*
369	11.5
534	21.5
737	40.7
414	12.6
1.31e+03	148
1.31e+03	129
912	68.3
367	11.3
1.14e+03	113
924	68.6
1.32e+03	142
876	61.9
964	84.6
641	32.6
597	28.6
1.23e+03	128
608	27
1.04e+03	83.7
1.17e+03	116
881	56
1.24e+03	124
945	77.5
1.04e+03	94.3
1.01e+03	77.5
374	12.1
1.37e+03	146
1.02e+03	88.7
1.02e+03	83.6
1.09e+03	119
1.13e+03	103
1.08e+03	87.6
379	11.9
1.06e+03	100
671	35.3
837	54
500	17.1
1.31e+03	132
952	77.3
1.12e+03	103
1.29e+03	127
1.09e+03	94.9
925	70.8
922	64.2
589	26.6
895	65.9
980	92.3
1.15e+03	107
1.08e+03	99.1
582	27.1
876	67.1
712	42.3
1.4e+03	153
460	13.6
1.27e+03	119
1.32e+03	153
933	68.1
756	50.4
1.24e+03	131
1.13e+03	103
1.04e+03	89.4
1.19e+03	122
1.34e+03	149
1.03e+03	95.6
458	11.9
1.2e+03	120
1.29e+03	131
843	53.4
1.39e+03	157
825	56.9
1.13e+03	110
648	33
1.14e+03	114
861	59.9
693	37
915	62.8
1.09e+03	94.1
1.25e+03	144
515	17.6
1.14e+03	112
1.12e+03	119
1.14e+03	103
1.31e+03	127
1.25e+03	143
1.22e+03	145
955	71.9
1.07e+03	100
1.16e+03	123
1.01e+03	84.4
842	56